In [1]:
#spacy
import spacy
import en_core_web_lg
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc, Span
from spacy.matcher import Matcher, PhraseMatcher


#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
#import pyLDAvis.gensim_models
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines
import json
import os
import csv

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

import time
#warning
import warnings 
warnings.filterwarnings('ignore')

2021-12-06 19:59:00.806394: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-06 19:59:00.806447: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lewagonlew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/lewagonlew/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
PATH = os.path.dirname(os.path.dirname(os.getcwd()))
PATH

## Creating Dictionary File from different files/formats (degrees.csv + new_dictionary.json)

In [ ]:
degrees = pd.read_csv(PATH + "/skills_matcher/data/dictionaries/degrees.csv")
degrees.shape

In [ ]:
degrees.rename(columns = {"Skill":"Pattern"})

In [ ]:
degree_list = degrees.values.tolist()

In [ ]:
#creating list from json line file (adding degrees.csv)

skill_pattern = []
with open('../data/dictionaries/new_dict.jsonl') as f:
    for line in f:
        skill_pattern.append(json.loads(line))

In [ ]:
len(skill_pattern)

In [ ]:
len(degree_list)

In [ ]:
list1 = []
for i in range(0, len(skills['Skill'])):
    dict1 = {}
    dict1['label'] = 'SKILL'
    skill = skills['Skill'][i]
    sub_dict = {}
    sub_dict['TEXT'] = skill
    dict1['pattern'] = [sub_dict]
    list1.append(dict1)
    

In [ ]:
## function to format dataframe to needed json format

def pd_to_json_format(df):    
    new_list = []
    new_skills = df
    
    for i in range(0, len(new_skills)):
        if new_skills["Label"][i] == "SKILL":
            dict1 = {}
            dict1['label'] = 'SKILL|'+ new_skills["Skill"][i].replace(' ', '-')
            skill = new_skills['Skill'][i].lower().split(' ')
            sub_dict = {}
            sub_list = []

            for item in skill:
                sub_dict['LOWER'] = item
                sub_list.append(sub_dict.copy())
                dict1['pattern'] = sub_list
            new_list.append(dict1)

        if new_skills["Label"][i] == "KNOWLEDGE":
            dict1 = {}
            sub_dict = {}
            sub_list = []
            dict1['label'] = 'KNOWLEDGE'

            sub_dict['LOWER'] = new_skills['Skill'][i]
            sub_list.append(sub_dict.copy())
            dict1['pattern'] = sub_list
            new_list.append(dict1)

        if new_skills["Label"][i] == "MIN_EXP":
            dict1 = {}
            sub_dict = {}
            sub_list = []
            dict1['label'] = 'MIN_EXP'

            sub_dict['LOWER'] = new_skills['Skill'][i]
            sub_list.append(sub_dict.copy())
            dict1['pattern'] = sub_list
            new_list.append(dict1)

        if new_skills["Label"][i] == "DEGREE":
            dict1 = {}
            sub_dict = {}
            sub_list = []
            dict1['label'] = 'DEGREE'

            sub_dict['LOWER'] = new_skills['Skill'][i]
            sub_list.append(sub_dict.copy())
            dict1['pattern'] = sub_list
            new_list.append(dict1)

    return new_list

In [ ]:
json_list = pd_to_json_format(degrees)

In [ ]:
skill_pattern.extend(json_list)

In [ ]:
## number of elements in entitydictionary 

len(skill_pattern)

In [3]:
import ujson
from pathlib import Path

def read_jsonl(file_path):
    """Read a .jsonl file and yield its contents line by line.
    file_path (unicode / Path): The file path.
    YIELDS: The loaded JSON contents of each line.
    """

    with Path(file_path).open('r', encoding='utf8') as f:
        for line in f:
            try:  # hack to handle broken jsonl
                yield ujson.loads(line.strip())
            except ValueError:
                continue


def write_jsonl(file_path, lines):
    """Create a .jsonl file and dump contents.
    file_path (unicode / Path): The path to the output file.
    lines (list): The JSON-serializable contents of each line.
    """
    data = [ujson.dumps(line, escape_forward_slashes=False) for line in lines]
    Path(file_path).open('w', encoding='utf-8').write('\n'.join(data))

In [ ]:
##Complete Dictionary which is used for spacy nlp entity_ruler

complete_dict_json_path = PATH + '/skills_matcher/data/dictionaries/complete_dict.jsonl'

write_jsonl(complete_dict_json_path, skill_pattern)

In [ ]:
complete_dict_json_path

## Importing scraped and pre-cleaned Job Description

In [ ]:
PATH

In [ ]:
skills_matcher/data/cleaned_data.csv

In [ ]:
#job description data

data = pd.read_csv('../skills_matcher/data/cleaned_data.csv')
data = data.drop(['Unnamed: 0'], axis = 1)
data = data.dropna()
data = data.drop_duplicates()
data.head()

In [ ]:
data = data.drop(columns = ["ISCO", "major_job", "location"])

In [ ]:
data.dropna()

In [ ]:
#mixing job description and taking smaller sample for faster computation

data = data.reindex(np.random.permutation(data.index))
data = data.copy().iloc[
    0:200,
]
data.head()

In [ ]:
data["description"].loc[0]

# different spacy models

In [ ]:
# spacy english model - change to small (en_core_web_lg) for faster computation - trained with skill_pattern.jsonl (Microsoft)

nlp_ms = spacy.load("en_core_web_sm")
skill_pattern_path =  "../data/skill_patterns.jsonl"

ruler = nlp_ms.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp_ms.pipe_names

In [ ]:
#2nd model trained with our skill_list

nlp_2 = spacy.load('en_core_web_sm')

ruler_2 = nlp_2.add_pipe("entity_ruler")

ruler_2.from_disk(new_dict_json_path)

nlp_2.pipe_names

## cleaning and skill extraction functions (not used)

In [ ]:
def get_skills(text):
    doc = nlp_ms(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [ ]:
#cleaning data function - call with TRUE value if lowering/stopword removiing/punctuation removing/ lemmatizing should be done
def clean_data(data, lower = False, stopw=False, punctuation=False, lemmatize=False):
    data = data.dropna()
    
    # digits
    #data['description'] = data['description'].str.replace('\d+', '')
        
    if lower == True:
            ## Lower case
            data['description'] = data['description'].apply(lambda x: " ".join(x.lower()for x in x.split()))
            
    ## remove tabulation and punctuation
    if punctuation == True:
            data['description'] = data['description'].str.replace('[^\w\s]',' ')
            
    ## stopwords
    if stopw == True:
            stop = stopwords.words('english')
            data['description'] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
            
    ## lemmatizing
    if lemmatize == True:
            lemmatizer = WordNetLemmatizer()
            data['description'] = data['description'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))

    ##depending on data frame, drop more columns, re-index
    ##clean_data.drop(columns = "position")
    
    return data

In [ ]:
clean_data = clean_data(data, True, True, True, True)

In [ ]:
clean_data.head()

In [ ]:
clean_data.shape

In [ ]:
"""get_skills/unique_skills couldn´t extract entities from doc - see new approach"""

clean_data["skills"] = clean_data["description"].str.lower().apply(get_skills)
clean_data["skills"] = clean_data["skills"].apply(unique_skills)


In [ ]:
clean_data.head()

In [ ]:
clean_data['description'].loc[1]

In [ ]:
doc = nlp_2("A complex-example,!")
print([token.text for token in doc])

In [ ]:
sent = nlp_2(clean_data["description"][109])
ents = [(ent.text, ent.label_) for ent in sent.ents]
displacy.render(sent, style="ent", jupyter=True)
print(ents)

In [ ]:
clean_data

In [ ]:
fig = px.histogram(
    clean_data, x="job", title="Distribution of Jobs Categories"
).update_xaxes(categoryorder="total descending")
fig.show()

In [ ]:
df["description"][216]

## Back to cleaned job-description Data

In [5]:
#functions for getting entities text, label, start and end of character position
def create_skill_set(doc):
    '''Create a set of the extracted skill entities of a doc'''
    #nlp_doc = nlp_ms(doc)
    return set([ent.label_.upper()[6:] for ent in doc.ents if 'skill' or 'knowledge' in ent.label_.lower()])

def create_skill_position(doc):
    '''Create a set of the extracted skill entities of a doc'''
    #nlp_doc = nlp_ms(doc)
    #[(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in sent.ents]
    return list([ent.start_char, ent.end_char, ent.label_] for ent in doc.ents )

#def create_skillset_dict(resume_texts):
#    '''Create a dictionary containing a set of the extracted skills. Name is key, matching skillset is value'''
#    skillsets = [create_skill_set(resume_text) for resume_text in resume_texts]
#   return skillsets


In [115]:
nlp_ms = spacy.blank("en")
skill_pattern_path =  "../skills_matcher/data/dictionaries/complete_dict.jsonl"

ruler = nlp_ms.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp_ms.pipe_names

['entity_ruler']

In [54]:
data = pd.read_csv('../skills_matcher/data/cleaned_data.csv')
data = data.drop(['Unnamed: 0'], axis = 1)
data = data.dropna()
data = data.drop_duplicates()
data.head()

,ISCO,major_job,job,position,location,description
0,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...
1,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Geneva,[' Be in charge of the development of applicat...
2,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Geneva,[' Contribute to the maintenance and developme...
3,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Medical Devices Physicist,"Newton, Cambridgeshire",[' Agency: Newton Colmore Consulting Reference...
4,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Fluidics Physicist,Cambridge,[' Agency: Newton Colmore Consulting Reference...


In [55]:
data = data.drop(columns = ["ISCO", "major_job", "location"])
data.dropna(inplace=True)
data.drop_duplicates(subset='description', keep="first", inplace = True)
data.reset_index(inplace = True)
data.drop(columns = "index")

,job,position,description
0,physicist,Accelerator Physicist id54315,[' You have an academic degree in physics or e...
1,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,[' Be in charge of the development of applicat...
2,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,[' Contribute to the maintenance and developme...
3,physicist,Medical Devices Physicist,[' Agency: Newton Colmore Consulting Reference...
4,physicist,Fluidics Physicist,[' Agency: Newton Colmore Consulting Reference...
...,...,...,...
3285,washer,Car Valeter / Detailer / Washer,"[' To wash, clean, valet and detail vehicles i..."
3286,washer,Detail Technician/Fleet Washer/Truck Washer/Au...,[' Paid Training Direct Care (health) Paid Vac...
3287,washer,Horeca Medewerker Afwas/Stewarding / Hospitali...,[' Enthousiast en energiek Eerlijk Flexibel in...
3288,washer,Dishwasher 50%,[' Du har minst 1 års erfaring som oppvaskhjel...


In [56]:
data.duplicated(['description']).sum()

0

In [14]:
#cleaning description string
def clean_string(df):
    for i in range (0, len(df)):
        #description = re.sub(
        #'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        #" ",
        #df.description.iloc[i],)
        description = df.description.iloc[i].lstrip('["').rstrip('"]').replace("'", "")
        
        description = re.sub(' {2,}', ' ', description)
        df['description'][i] = description.rstrip(" ")

    return df

In [57]:
#data["cleaned_text"] = data["description"].apply(clean_string)
data = clean_string(data)
data.description[3183]

' All daily nanny duties Helping achieve development & Milestones by planning activities Organising playdates, playgroups and going to the park Babysitting Housekeeping in relation to the children Preparing healthy meals and snacks , Ideally looking for a long term nanny Good English communication skills Someone who loves working with children and planning activities'

In [58]:
data.drop_duplicates(subset='description', keep="first", inplace = True)
data

,index,job,position,description
0,0,physicist,Accelerator Physicist id54315,You have an academic degree in physics or eng...
1,1,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Be in charge of the development of applicatio...
2,2,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Contribute to the maintenance and development...
3,3,physicist,Medical Devices Physicist,Agency: Newton Colmore Consulting Reference: ...
4,4,physicist,Fluidics Physicist,Agency: Newton Colmore Consulting Reference: ...
...,...,...,...,...
3285,4403,washer,Car Valeter / Detailer / Washer,"To wash, clean, valet and detail vehicles ins..."
3286,4404,washer,Detail Technician/Fleet Washer/Truck Washer/Au...,Paid Training Direct Care (health) Paid Vacat...
3287,4405,washer,Horeca Medewerker Afwas/Stewarding / Hospitali...,Enthousiast en energiek Eerlijk Flexibel inze...
3288,4406,washer,Dishwasher 50%,Du har minst 1 års erfaring som oppvaskhjelp ...


In [17]:
data.shape

(3290, 4)

In [60]:
data.description.loc[data.description.str.startswith(' Lead, motivate and coordinate the duties', na=False)]

2880     Lead, motivate and coordinate the duties of 5...
Name: description, dtype: object

In [61]:
data.description[2880]

' Lead, motivate and coordinate the duties of 5 staff of the Finance function with responsibilities for all related transactions for: general ledger postings, accounts payable function, accounts receivable function, fixed asset ledger and cash and bank. Daily Cash flow management and weekly projections. Determines proper handling of all financial transactions and manages timely monthly close to ensure accurate Trial Balance, Profit centre reporting and variance analysis. Monitors compliance with International Financial and Reporting Standards (IFRS), company procedures and legal requirements. Preparation and submission of Quarterly VAT returns. Co-ordinates the annual independent audit, being the key contact with auditors, including all working papers, reconciliations and supporting schedules. Provides required information and analysis to auditors for all Tax submissions. Oversees internal control systems for all departments to ensure safeguarding of company assets. Responsible for hol

In [81]:
%%time
#applying spacy model to create entity marked/structured text
data["marked_text"] = data["description"].str.lower().apply(nlp_ms)
data.head()

CPU times: user 20.2 s, sys: 59.6 ms, total: 20.2 s
Wall time: 20.2 s


,job,position,description,marked_text,skills_pos
0,physicist,Accelerator Physicist id54315,You have an academic degree in physics or eng...,"( , you, have, an, academic, degree, in, physi...","[[43, 54, SKILL|engineering], [266, 279, SKILL..."
1,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Be in charge of the development of applicatio...,"( , be, in, charge, of, the, development, of, ...","[[48, 56, SKILL|software], [107, 128, SKILL|di..."
2,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Contribute to the maintenance and development...,"( , contribute, to, the, maintenance, and, dev...","[[271, 278, SKILL|support], [473, 479, SKILL|m..."
3,physicist,Medical Devices Physicist,Agency: Newton Colmore Consulting Reference: ...,"( , agency, :, newton, colmore, consulting, re...",[]
4,physicist,Fluidics Physicist,Agency: Newton Colmore Consulting Reference: ...,"( , agency, :, newton, colmore, consulting, re...",[]


In [ ]:
#applying create_skill_set/position
data["skills_pos"] = data["marked_text"].apply(create_skill_position)
data.head()

In [146]:
#creating skill set
data["skills"] = data["marked_text"].apply(create_skill_set)

In [150]:
data["skills"][100:200]

100                           {MEDIUM}
101                                {C}
102                          {VERILOG}
103                      {PYTHON, C++}
104                        {PYTHON, C}
                    ...               
195                    {LINUX, VMWARE}
196                            {REACT}
197                           {VMWARE}
198    {LINUX, JAVA, UNIX, DEPLOYMENT}
199                      {LINUX, BASH}
Name: skills, Length: 100, dtype: object

In [147]:
data.shape

(660, 6)

In [65]:
data.drop(columns="index", inplace = True)

In [83]:
data.head()

,job,position,description,marked_text,skills_pos
0,physicist,Accelerator Physicist id54315,You have an academic degree in physics or eng...,"( , you, have, an, academic, degree, in, physi...",[]
1,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Be in charge of the development of applicatio...,"( , be, in, charge, of, the, development, of, ...","[[682, 688, SKILL|python], [992, 998, SKILL|py..."
2,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Contribute to the maintenance and development...,"( , contribute, to, the, maintenance, and, dev...","[[473, 479, SKILL|medium]]"
3,physicist,Medical Devices Physicist,Agency: Newton Colmore Consulting Reference: ...,"( , agency, :, newton, colmore, consulting, re...",[]
4,physicist,Fluidics Physicist,Agency: Newton Colmore Consulting Reference: ...,"( , agency, :, newton, colmore, consulting, re...",[]


In [95]:
#check how many empty lists (e.g. model didn´t find matching pattern) there are
data.skills_pos[data.skills_pos.str.len() != 0]

data = data[data.skills_pos.str.len() != 0]

In [102]:
data.reset_index(inplace = True)
data.drop(columns = "index", inplace = True)
data

,level_0,job,position,description,marked_text,skills_pos
0,0,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Be in charge of the development of applicatio...,"( , be, in, charge, of, the, development, of, ...","[[682, 688, SKILL|python], [992, 998, SKILL|py..."
1,1,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Contribute to the maintenance and development...,"( , contribute, to, the, maintenance, and, dev...","[[473, 479, SKILL|medium]]"
2,2,physicist,Physicist/Optical Scientist/Engineer Technolog...,"PhD in physics or engineering (optical, micro...","( , phd, in, physics, or, engineering, (, opti...","[[477, 478, SKILL|C], [479, 482, SKILL|C++]]"
3,3,physicist,Experimental Physicist,You will have experience with microfluidic ch...,"( , you, will, have, experience, with, microfl...","[[172, 178, SKILL|Python]]"
4,4,physicist,Experimental Physicist,You will have experience with microfluidic ch...,"( , you, will, have, experience, with, microfl...","[[172, 178, SKILL|Python]]"
...,...,...,...,...,...,...
655,655,cleaner,cleaner,Weekly pay - £9.50 p/h Holiday Pay Guaranteed...,"( , weekly, pay, -, £, 9.50, p, /, h, holiday,...","[[126, 128, SKILL|Go]]"
656,656,cleaner,Cleaner,Treat yourself and your friends with 50% Zizz...,"( , treat, yourself, and, your, friends, with,...","[[100, 102, SKILL|Go]]"
657,657,cleaner,Cleaner,"High and low dusting, mopping, wiping and scr...","( , high, and, low, dusting, ,, mopping, ,, wi...","[[997, 999, SKILL|Go]]"
658,658,cleaner,Cleaner,Cleaning Experience in an commercial environm...,"( , cleaning, experience, in, an, commercial, ...","[[244, 246, SKILL|Go]]"


In [103]:
data.drop(columns = "level_0", inplace = True)
data

,job,position,description,marked_text,skills_pos
0,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Be in charge of the development of applicatio...,"( , be, in, charge, of, the, development, of, ...","[[682, 688, SKILL|python], [992, 998, SKILL|py..."
1,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Contribute to the maintenance and development...,"( , contribute, to, the, maintenance, and, dev...","[[473, 479, SKILL|medium]]"
2,physicist,Physicist/Optical Scientist/Engineer Technolog...,"PhD in physics or engineering (optical, micro...","( , phd, in, physics, or, engineering, (, opti...","[[477, 478, SKILL|C], [479, 482, SKILL|C++]]"
3,physicist,Experimental Physicist,You will have experience with microfluidic ch...,"( , you, will, have, experience, with, microfl...","[[172, 178, SKILL|Python]]"
4,physicist,Experimental Physicist,You will have experience with microfluidic ch...,"( , you, will, have, experience, with, microfl...","[[172, 178, SKILL|Python]]"
...,...,...,...,...,...
655,cleaner,cleaner,Weekly pay - £9.50 p/h Holiday Pay Guaranteed...,"( , weekly, pay, -, £, 9.50, p, /, h, holiday,...","[[126, 128, SKILL|Go]]"
656,cleaner,Cleaner,Treat yourself and your friends with 50% Zizz...,"( , treat, yourself, and, your, friends, with,...","[[100, 102, SKILL|Go]]"
657,cleaner,Cleaner,"High and low dusting, mopping, wiping and scr...","( , high, and, low, dusting, ,, mopping, ,, wi...","[[997, 999, SKILL|Go]]"
658,cleaner,Cleaner,Cleaning Experience in an commercial environm...,"( , cleaning, experience, in, an, commercial, ...","[[244, 246, SKILL|Go]]"


In [ ]:
def convert(o):
    if isinstance(o, numpy.int64): return int(o)  
    raise TypeError

json.dumps({'value': numpy.int64(42)}, default=convert)

,job,position,description,marked_text,skills_pos
0,physicist,Accelerator Physicist id54315,You have an academic degree in physics or eng...,"( , you, have, an, academic, degree, in, physi...",[]
1,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Be in charge of the development of applicatio...,"( , be, in, charge, of, the, development, of, ...","[[682, 688, SKILL|python], [992, 998, SKILL|py..."
2,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Contribute to the maintenance and development...,"( , contribute, to, the, maintenance, and, dev...","[[473, 479, SKILL|medium]]"
3,physicist,Medical Devices Physicist,Agency: Newton Colmore Consulting Reference: ...,"( , agency, :, newton, colmore, consulting, re...",[]
4,physicist,Fluidics Physicist,Agency: Newton Colmore Consulting Reference: ...,"( , agency, :, newton, colmore, consulting, re...",[]
...,...,...,...,...,...
3285,washer,Car Valeter / Detailer / Washer,"To wash, clean, valet and detail vehicles ins...","( , to, wash, ,, clean, ,, valet, and, detail,...",[]
3286,washer,Detail Technician/Fleet Washer/Truck Washer/Au...,Paid Training Direct Care (health) Paid Vacat...,"( , paid, training, direct, care, (, health, )...",[]
3287,washer,Horeca Medewerker Afwas/Stewarding / Hospitali...,Enthousiast en energiek Eerlijk Flexibel inze...,"( , enthousiast, en, energiek, eerlijk, flexib...",[]
3288,washer,Dishwasher 50%,Du har minst 1 års erfaring som oppvaskhjelp ...,"( , du, har, minst, 1, års, erfaring, som, opp...",[]


# creating annotation_data json file format

In [ ]:
data.head(100)

In [96]:
def pd_to_json_format(df):    
    new_list = []
    
    for i in range (0, len(df)):
        dict1 = {}
        dict1['ID'] = int(df.index[i])
        dict1['data'] = df.description[i]
        dict1['label'] = df.skills_pos[i]
        new_list.append(dict1.copy())
        
    return new_list

In [104]:
json_list = pd_to_json_format(data)

In [123]:
json_list[400]

{'ID': 400,
 'data': ' Location: Gdańsk POL Start Date: 2022-01-17 Salary (gross): 10.0k-20.0k (UoP) PLN / month Category: Fullstack Company Size: 20+ , C++ English Linux Git Python Proactivity Problem solving Open CV Image processing 3D processing , HALCON VISION LIBRARY Docker Leadership skills Critical thinking ROS Robot Operating System',
 'label': [[131, 134, 'SKILL|C++'],
  [143, 148, 'SKILL|Linux'],
  [149, 152, 'SKILL|Git'],
  [153, 159, 'SKILL|Python'],
  [251, 257, 'SKILL|docker']]}

In [105]:
path =  '../skills_matcher/data/jd_auto_annotation.jsonl'
write_jsonl(path, json_list)

In [ ]:
##creating json_file from list (in json format), saving it to disk

jsonStr = json.dumps(json_list)
jsonFile = open("JD_annotation_data.json", "w")
jsonFile.write(jsonStr)
jsonFile.close()

# End of Base_model Notebook

#### adding Job Category as Entity

In [ ]:
df.Category.unique()

In [ ]:
patterns = df.Category.unique()
for a in patterns:
    ruler.add_patterns([{"label": "Job-Category", "pattern": a}])

In [ ]:
#options= {"ents": "Job-Category", "colors": "#ff3232"},{"ents": "SKILL", "colors": "#56c426"}
colors = {
    "Job-Category": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
    "SKILL": "linear-gradient(90deg, #9BE15D, #00E3AE)",
    "ORG": "#ffd966",
    "PERSON": "#e06666",
    "GPE": "#9fc5e8",
    "DATE": "#c27ba0",
    "ORDINAL": "#674ea7",
    "PRODUCT": "#f9cb9c",
}
options = {
    "ents": [
        "Job-Category",
        "SKILL",
        "ORG",
        "PERSON",
        "GPE",
        "DATE",
        "ORDINAL",
        "PRODUCT",
    ],
    "colors": colors,
}
sent = nlp_2(df["Resume_str"].iloc[99])
displacy.render(sent, style="ent", jupyter=True, options=options)

# Visualization/Matching 

In [143]:
test_resume = nlp_ms(df["Resume_str"].iloc[0])
resume_ents = [(ent.text, ent.label_) for ent in test_resume.ents]
#displacy.render(sent, style="ent", jupyter=True)
print(resume_ents)

[('communications', 'SKILL|communications'), ('publishing', 'SKILL|publishing'), ('Marketing', 'SKILL|marketing'), ('publishing', 'SKILL|publishing'), ('marketing', 'SKILL|marketing'), ('chef', 'SKILL|Chef'), ('email marketing', 'SKILL|email-marketing'), ('MailChimp', 'SKILL|mailchimp'), ('business', 'SKILL|business'), ('support', 'SKILL|support'), ('marketing', 'SKILL|marketing'), ('Communications', 'SKILL|communications'), ('Communications', 'SKILL|communications'), ('communications', 'SKILL|communications'), ('Communications', 'SKILL|communications'), ('communications', 'SKILL|communications'), ('software', 'SKILL|software'), ('communications', 'SKILL|communications'), ('marketing', 'SKILL|marketing'), ('Google analytics', 'SKILL|google-analytics'), ('business', 'SKILL|business'), ('support', 'SKILL|support'), ('marketing', 'SKILL|marketing'), ('graphic design', 'SKILL|graphic-design'), ('design', 'SKILL|design'), ('communications', 'SKILL|communications'), ('email marketing', 'SKIL

In [142]:
resume_2 = "HR ASSISTANT/PAYROLL         Highlights        Paychex, Mastertax, ABRA, Time Star, InfoTronics, Store Master, Dominion, QuickBooks\nCeridian, PeopleSoft, SAP, Microsoft Word, Excel and PowerPoint, Crystal Reports              Experience      HR Assistant/Payroll    Company Name   －   City  ,   State      Human Resources professional offering over 15 years of combined HR and payroll\n          experience in both manufacturing and academic settings\nAccurate and timely preparation of employee payroll, payroll tax information, and\n          employee benefits administration\nHighly effective employer representation in unemployment claims and hearings.         01/2014   to   01/2014     Payroll & Benefits Coordinator    Company Name   －   City  ,   State      Process weekly payroll for 100 hourly and salaried Associates\n    Garnishment, Levy and Child Support processing\n    Compile weekly labor hours      \n    Enter new hire information into HRIS/Payroll system\n    Assist with Annual Benefits Open Enrollment\n    Process employee status changes in payroll, personnel, and benefit records\n   Respond to Unemployment claims \n    Establish written policies and procedures\n    Assist in employee conflict resolution\n    Worker's Compensation accident reporting\n    Maintain Company-wide OSHA 300 and OSHA 300A.         01/2013   to   01/2013     Assistant Office Manager    Company Name   －   City  ,   State      Calculate daily timesheets \n     Enter new hire information into HRIS/Payroll system\n     Process employee status changes \n     Respond to Unemployment claims\n    Maintain OSHA 300 and OSHA 300A\n     Worker's Compensation accident reporting, loss time tracking\n    Process monthly billing\n    Process customer payments\n    Generate purchase orders\n    Maintain HRIS/Payroll system \n    Process invoices for payment.         01/2012   to   01/2013     HR Assistant/Payroll    Company Name   －   City  ,   State      Process weekly payroll for 100 hourly and salaried Associates\n    Garnishment, Levy and Child Support processing\n    Compile weekly labor hours      \n    Enter new hire information into HRIS/Payroll system\n    Assist with Annual Benefits Open Enrollment\n    Process employee status changes in payroll, personnel, and benefit records\n   Respond to Unemployment claims \n    Establish written policies and procedures\n    Assist in employee conflict resolution\n    Reviewed applications and identified potential candidates for position openings\n    Worker's Compensation accident reporting\n    Maintain Company-wide MSDS documents, OSHA 300 and OSHA 300A\n    Coordinate quarterly meetings.         01/2009   to   01/2012     Safety & Benefits Coordinator    Company Name   －   City  ,   State      promoted to this position in 2009)\n     Determine monthly benefit eligibility\n     Maintain benefit HRIS system\n     Determine quarterly 401k eligibility\n     Monthly audit of medical, dental and voluntary insurance products\n     Process 401k Benefit Event Notices \t\n     Record Associate and customer accidents\n     Maintain OSHA 300 log and annual OSHA 300A for multiple locations\n     Maintain Company-wide MSDS documents\n     Worker's Compensation and General Liability accident reporting and investigation \n     Conducted annual safety training\n     Coordinate monthly random drug screens\n     Respond to Unemployment claims, and effectively represent WESCO in Unemployment hearings\n     Assist with Annual Benefits Open Enrollment\n     Coordinate annual Recognition Luncheon\n     Assist in employee conflict resolution \n     Establish written policies and procedures.         01/2004   to   01/2009     Payroll / Human Resources Assistant    Company Name   －   City  ,   State      Process weekly and biweekly payroll for over 300 hourly and salaried Associates\n      Garnishment, Levy and Child Support processing\n      Compile weekly labor hours      \n      Enter new hire information into HRIS/Payroll system\n      Process employee status changes in payroll, personnel, and benefit records\n      Compilation and distribution of performance reviewsto multiple locations \n      Transfer weekly payroll direct deposit file \n      Process weekly 401k payments with outside vendor\n      Timely and accurate reporting of new hires to the State of MI\n      Respond to Unemployment claims, and effectively represent WESCO in Unemployment hearings.         01/2002   to   01/2004     Human Resources / Payroll Assistant    Company Name   －   City  ,   State      Primary contact between Human Resources and Corporate Payroll\n     Prepare and distribute disciplinary actions for unionized manufacturing plant\n     Maintain daily attendance records\n     Reviewed applications and identified potential candidates for position openings\n     Assist with generating Union layoffs and recalls\t\n     Maintain accurate confidential employee files\t     \n     Coordinate Annual Service Award Program.         01/2001   to   01/2002     Payroll Tax Analyst    Company Name   －   City  ,   State      Computed over 200 quarterly unemployment tax returns for every state in the Continental U.S.  Compiled withholding information for weekly tax deposits made via EFT\n     Examined tax information to file monthly and quarterly withholding returns for states and localities\n     Compiled divisional information to complete quarterly and annual Federal filings\n     Analyzed information for annual reporting to Federal, State and local government.         01/1999   to   01/2001     Payroll Clerk / Human Resources Assistant    Company Name   －   City  ,   State      Compiled information to calculate bi-weekly payroll for over 200 employees\n     Prepared information for quarterly Federal tax returns, sales and use tax, State and local reporting\n     Coordinated open enrollment for health care benefits\n     Ensured correct paperwork was timely filed for Workers Compensation claims\n     Researched employee records to analyze unemployment eligibility\n     Calculated field trip information for billing back to classrooms\n     Confirmed retirement information for monthly reporting to the State of Michigan\n     Maintained direct deposit system.         01/1996   to   01/1999     Payroll Clerk / Human Resources Assistant    Company Name   －   City  ,   State      Calculated weekly payroll for over 350 employees\n     Determined eligibility for health care benefits company-wide, including accurate maintenance of information\nLiaison between 400 employees and health care insurance company\nMaintained necessary materials required to conduct new hire orientations\nReviewed applications and identified potential candidates for position openings\nResearched employee information to complete unemployment determination\nFiled New Hire reports to meet State compliance\nReviewed weekly uniform billing for 200 employees, including verification of correct billing and submitted credits\nDeveloped spreadsheets to track employee reviews, insurance, and holiday pay\nEnabled accurate payroll information to be extracted from spreadsheets.          Education          B.B.A  :   Human Resources    Human Resources            MIOSHA Level One Certification\n          Baker College          General Industry Certification      Macomb Community College   －   City  ,   State              Skills    ABRA, academic, Benefits, benefits administration, billing, bi, Ceridian, conflict resolution, Crystal Reports, HRIS, Human Resources, HR, insurance, Process invoices, local government, materials, meetings, Excel, PowerPoint, Microsoft Word, Payroll, PeopleSoft, performance reviews, personnel, policies, QuickBooks, reporting, safety, sales, SAP, spreadsheets, tax, tax retur"

In [145]:
test_resume_2 = nlp_ms(resume_2)
resume_2_ents = [(ent.text, ent.label_) for ent in test_resume_2.ents]
displacy.render(sent, style="ent", jupyter=True)
print(resume_2_ents)

[('Ceridian', 'SKILL|Ceridian'), ('SAP', 'SKILL|SAP'), ('Crystal', 'SKILL|crystal'), ('Support', 'SKILL|support'), ('time tracking', 'SKILL|time-tracking'), ('payments', 'SKILL|payments'), ('Support', 'SKILL|support'), ('Support', 'SKILL|support'), ('payments', 'SKILL|payments'), ('Ceridian', 'SKILL|Ceridian'), ('Crystal', 'SKILL|crystal'), ('SAP', 'SKILL|SAP')]


In [151]:
matching_skills = data["skills"]

In [152]:
matching_skills

0      {PYTHON}
1      {MEDIUM}
2      {C, C++}
3      {PYTHON}
4      {PYTHON}
         ...   
655        {GO}
656        {GO}
657        {GO}
658        {GO}
659        {GO}
Name: skills, Length: 660, dtype: object

In [ ]:
req_skills = input_skills.lower().split(",")
resume_skills = unique_skills(get_skills(input_resume.lower()))
score = 0
for x in req_skills:
    if x in resume_skills:
        score += 1
req_skills_len = len(req_skills)
match = round(score / req_skills_len * 100, 1)

print(f"The current Resume is {match}% matched to your requirements")

In [ ]:
def match_skills(JD_set, cv_set):
    '''Get intersection of resume skills and job offer skills and return match percentage'''
    
    if len(cv_set) < 1:
        print('could not extract skills from resume text')   
    else:
        for i in range(0, len(JD_set))
        
        
        #implement a function comparing with a list of job_descriptions 
        pct_match = round(len(JD_set.intersection(cv_set[resume_name])) / len(JD_set) * 100, 0)
        print("Resume has a {}% skill match on job offer".format(pct_match))
        print('Required skills: {} '.format(JD_set))
        print('Matched skills: {} \n'.format(JD_set.intersection(skillset_dict[resume_name])))
        
        return (resume_name, pct_match)

In [ ]:
# example of job offer text (string). Can input your own.
vacature_text = vacatures_df[vacatures_df['soort_vacature'] == 'Data Scientist'].skills.iloc[13]

# Create a set of the skills extracted from the job offer text
vacature_skillset = create_skill_set(nlp(vacature_text))

# Create a list with tuple pairs containing the names of the candidates and their match percentage
match_pairs = [match_skills(jd_skillset, skillset_dict, ) for name in skillset_dict.keys()]

In [ ]:
# Sort tuples from high to low on match percentage
match_pairs.sort(key=lambda tup: tup[1], reverse=True)

# Unpack tuples
names, pct = zip(*match_pairs)

# Plotting
sns.set_theme(style='darkgrid')

fig, ax = plt.subplots(figsize=(12,8))
ax.set_title('Job offer match with candidates', fontsize=20)
ax.set(xlabel='Candidates', ylabel='% Match')
ax.set(ylim=(0, 100))


sns.set(font_scale=1.5)
plot = sns.barplot(x=list(names), y=list(pct), color='b')

plt.show()
fig = plot.get_figure()
fig.savefig('barplot.png') 